In [11]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from tensorflow.keras.models import model_from_json, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import ResNet50 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

from skimage.transform import rescale, resize, downscale_local_mean

In [12]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    if ds.Modality == 'DX':
        if ds.PatientPosition in ['PA', 'AP']:
            if ds.BodyPartExamined == 'CHEST':
                return img
            else:
                print('Image has invalid body part details. It is not a Chest body part')
                return None 
        else:
            print('Image has invalid Xray view position data')
            return None 
    else:
        print('Image has invalid modality data. It does not have value as DX')
        return None 
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input

# def preprocess_image(img,img_mean,img_std,img_size): 

def preprocess_image(img,img_size): 
    img = resize(img, (img_size[1], img_size[2]))
    proc_img = (img - img.mean()) / img.std()
    return np.resize(proc_img, img_size)

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    with open(model_path, "r") as file:
        loaded_model = file.read()
    model = model_from_json(loaded_model)
    model.load_weights(weight_path)
    
    return model
    

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model

def predict_image(model, img, thresh): 
    prediction = model(img)
    if(prediction > thresh):
        prediction = 'Pneumonia'
    else:
        prediction = "No Pneumonia"
    
    return prediction


In [13]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "Pneumonia_Classifier.json" #path to saved model
weight_path = "xray_class_new_model.best.hdf5" #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
# img_mean = # loads the mean image value they used during training preprocessing
# img_std = # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.58 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
  #  img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    img_proc = preprocess_image(img,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
No Pneumonia
Load file test2.dcm ...
Pneumonia
Load file test3.dcm ...
Pneumonia
Load file test4.dcm ...
Image has invalid body part details. It is not a Chest body part
Load file test5.dcm ...
Image has invalid modality data. It does not have value as DX
Load file test6.dcm ...
Image has invalid Xray view position data
